***Webscraper Hotels Data - 123 Milhas / Decolar***

*   Data extraction name, price and provider hotels website 123 Milhas
*   Data extraction name and price hotels website Decolar
*   Report rate comparison

In [1]:
!pip install selenium
!pip install unidecode
!pip install pyjarowinkler
!apt-get update 
!apt install chromium-chromedriver

     |████████████████████████████████| 958 kB 26.5 MB/s 
     |████████████████████████████████| 138 kB 51.8 MB/s 
     |████████████████████████████████| 356 kB 43.7 MB/s 
     |████████████████████████████████| 3.6 MB 56.9 MB/s 
     |████████████████████████████████| 55 kB 3.1 MB/s 
     |████████████████████████████████| 58 kB 6.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.8 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 235 kB 9.4 MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bioni

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import *
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from unidecode import unidecode
from time import sleep
import pandas as pd
import os

In [3]:
# ID destinations

# 123 Milhas
  # Aracaju: 
  # Arraial dAjuda: 
  # Balneario Camboriu: 
  # Belo Horizonte: 
  # Brasilia: 
  # Buzios: 
  # Cabo Frio: 
  # Caldas Novas: 
  # Campinas: 
  # Campos do Jordao: 
  # Canela: 
  # Curitiba: 
  # Florianopolis: 1001872
  # Fortaleza: 
  # Foz do Iguacu: 
  # Gramado: 
  # Ilhabela: 
  # Ilheus: 
  # Joao Pessoa: 
  # Maceio: 
  # Morro de Sao Paulo: 
  # Natal: 
  # Porto Alegre: 
  # Porto de Galinhas: 
  # Porto Seguro: 
  # Praia de Pipa: 
  # Recife: 
  # Rio de Janeiro: 
  # Salvador: 
  # Sao Paulo: 

# Decolar
  # Aracaju: 180
  # Arraial dAjuda: 339
  # Balneario Camboriu: 192594
  # Belo Horizonte: 701
  # Brasilia: 926
  # Buzios: 1077
  # Cabo Frio: 1227
  # Caldas Novas: 1364
  # Campinas: 1467
  # Campos do Jordao: 6116
  # Canela: 6064
  # Curitiba: 1595
  # Florianopolis: 2261
  # Fortaleza: 2302
  # Foz do Iguacu: 3072
  # Gramado: 2611
  # Ilhabela: 3027
  # Ilheus: 3158
  # Joao Pessoa: 3399
  # Maceio: 4430
  # Morro de Sao Paulo: 4678
  # Natal: 4971
  # Porto Alegre: 5822
  # Porto de Galinhas: 5648
  # Porto Seguro: 881
  # Praia de Pipa: 5875
  # Recife: 6322
  # Rio de Janeiro: 6381
  # Salvador: 7018
  # Sao Paulo: 6574

In [4]:
# Search params

name_destination = 'Florianopolis'
id_destination_123milhas = '1001872'
id_destination_decolar = '2261'
checkin = '06-02-2022' # Format DD-MM-AAAA
checkout = '08-02-2022' # Format DD-MM-AAAA
pax = 2 # Number of guests (adults)

In [5]:
# Chromedriver config

chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])
chrome_options.add_argument('--lang=pt-BR')
chrome_options.add_argument('--disable-notifications')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [6]:
# Get fee 123 Milhas

def get_fee():
    driver = webdriver.Chrome(options=chrome_options)
    link = f'https://123milhas.com/hoteis?adultos={pax}&bebes=0&checkin={checkin}&checkout={checkout}&criancas=0&id={id_destination_123milhas}&localizacao={name_destination},%20,%20Brasil&quartos={pax}&tipo=City&search_id='
    driver.get(link)
    sleep(3)
       
    price_net_temp = []
    tax_temp = []

    wait = WebDriverWait(driver, 10)

    wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'image-slider__main-image')))        
    hotel1_attributes = driver.find_element(By.XPATH, '/html/body/div[1]/div/hotel-card-price-list-and-filter-wrapper/div/div[3]/div[1]/hotel-list-card[1]/div[2]/div[3]/div[4]/theme-button/button').get_attribute('outerHTML')
    attributes_filter1 = ''.join(hotel1_attributes.partition('id')[1:])
    attributes_filter2 = attributes_filter1.find('type')
    attributes_filter3 = attributes_filter1[:attributes_filter2]
    attributes_replacers = attributes_filter3.replace('"','').replace('=','-').replace(' ','').replace('id-','')
    query = "'#" + attributes_replacers + "'"
    
    driver.execute_script(f"document.querySelector({query}).click();")
    new_window = driver.window_handles[1]
    driver.switch_to.window(new_window)
    
    wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'rooms-details-and-price__button')))
    driver.execute_script("document.querySelector('#hotel-details-and-info > div > rooms-details-and-price > div > div.rooms-details-and-price__prices-select-holder > theme-button > button').click();")
    
    wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'hotel-checkout-prices-and-tax')))        
    price_net_elem = driver.find_element(By.XPATH, '//*[@id="checkout-form"]/div/div[2]/div[1]/hotel-checkout-prices-and-tax/div/div[2]/span[2]')
    tax_elem = driver.find_element(By.XPATH, '//*[@id="checkout-form"]/div/div[2]/div[1]/hotel-checkout-prices-and-tax/div/div[3]/span[2]')

    price_net_temp.append(price_net_elem.text)
    tax_temp.append(tax_elem.text)
    price_net = rate_fmt(price_net_temp)
    tax = rate_fmt(tax_temp)
    
    global fee
    calc_fee = tax / price_net
    fee = float("%.4f" % calc_fee)
    return fee


def rate_fmt(rate):
    rate = ' '.join(rate)
    rate = rate.strip('R$').lstrip().replace(',','.')
    rate = float(rate)
    return rate



In [7]:
print('Fee:')
get_fee()

Fee:


0.1446

In [8]:
# Scraper Hotel Prices 123 Milhas

list_name_hotels_123milhas = []
list_price_hotels_123milhas = []
list_channel_hotels_123milhas = []

class Scraper123milhas:

    def scraper(self):
        self.data_collect()

    def data_collect(self):
        self.list_names = []
        self.list_prices = []
        self.list_channels = []
        self.driver = webdriver.Chrome(options=chrome_options)
        self.link = f'https://123milhas.com/hoteis?adultos={pax}&bebes=0&checkin={checkin}&checkout={checkout}&criancas=0&id={id_destination_123milhas}&localizacao={name_destination},%20,%20Brasil&quartos={pax}&tipo=City&search_id='
        self.driver.get(self.link)
        sleep(3)
        wait = WebDriverWait(self.driver, 10)

        for p in range(20): # Number of pages to browse (nº > total, run through all)
            item = 1
            wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'hotel-list-card__total-price')))
            list_length = self.driver.find_elements(By.CLASS_NAME, 'hotel-list-card__total-price')
            list_max = len(list_length)
            for i in range(list_max):
                c = 1
                while c < list_max:
                    try:
                        names_temp = self.driver.find_elements(By.XPATH,
                            f'//*[@id="hotelListHolder"]/div[3]/div[1]/hotel-list-card[{item}]/div[2]/div[1]/div/h4')
                        self.list_names.append(names_temp[0].text)                       
                        sleep(1)
                        prices_temp = self.driver.find_elements(By.XPATH,
                            f'//*[@id="hotelListHolder"]/div[3]/div[1]/hotel-list-card[{item}]/div[2]/div[2]/div[1]/div/div')
                        self.list_prices.append(prices_temp[0].text)          
                        sleep(1)
                        channels_temp = self.driver.find_element(By.XPATH,
                            f'/html/body/div[1]/div/hotel-card-price-list-and-filter-wrapper/div/div[3]/div[1]/hotel-list-card[{item}]/div[2]/div[1]/image-slider/div/div[1]/img').get_attribute('outerHTML')
                        if len(channels_temp) == 0:
                            channels_temp = self.driver.find_element(By.XPATH, f'/html/body/div[1]/div/hotel-card-price-list-and-filter-wrapper/div/div[3]/div[1]/hotel-list-card[{item}]/div[2]/div[2]/image-slider/div/div[1]/img').get_attribute('outerHTML')                        
                        if 'omnibees' in channels_temp:
                            self.list_channels.append(1)
                        else:
                            self.list_channels.append(0)
                        sleep(1)
                        item += 1
                    except:
                        item += 1
                        c += 1                        
            try:
              wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'pagination-next')))
              self.driver.execute_script('document.querySelectorAll("#hotelListHolder > div.hotel-card-price-list__cards-holder.hotel-card-price-list__obj-align > div.hotel-card-price-list__pagination-container > ul > li.pagination-next > a")[0].click();')
              print(f'\u001b[32m{"Browsing..."}\u001b[0m')
              sleep(3)
            except:
              self.driver.quit()

        print('Raw Data:')
        print(self.list_names)
        print(self.list_prices)
        print(self.list_channels)

        for name in self.list_names:
            list_name_hotels_123milhas.append(name)
        for price in self.list_prices:
            list_price_hotels_123milhas.append(price)  
        for channel in self.list_channels:
            list_channel_hotels_123milhas.append(channel)

start = Scraper123milhas()
start.scraper()


Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Raw Data:
['Hotel Cecomtur Florianópolis - Rede Andrade', 'Hotel Intercity Portofino Florianópolis', 'Hotel Valerim Florianópolis', 'Slaviero Baía Norte Florianópolis', 'Faial Prime Suites', 'Castelmar Hotel', 'Blue Tree Premium Florianópolis', 'Novotel Florianópolis', 'Cris Hotel', 'Village Paraíso', 'Lumar Hotel - Rede Andrade', 'Gentil Hotel', 'Hotel Intercity Florianópolis', 'Ingleses Praia Hotel', 'Porto da Ilha Hotel', 'Interclass Florianópolis', 'Iate Hotel Centro Florianópolis', 'Rio Branco Apart Hotel', 'Hotel Porto Sol Beach', 'Maria do Mar Hotel', 'Hotel Boutique Quinta das Videiras', 'Oscar Hotel', 'Hotel Palace 1', 'Jurerê Beach Village', 'Apart Hotel Roberto Monteiro', 'Ingleses Park Hotel', 'IL Campanario Villaggio Resort', 'WK Design 

In [9]:
# Data cleaning

name_hotels_123m_temp = list(pd.Series(list_name_hotels_123milhas).str.upper())
name_hotels_123m_fmtg = []

for name in name_hotels_123m_temp:
    name_hotels_123m_fmtg.append(unidecode(name))
print(name_hotels_123m_fmtg)

['HOTEL CECOMTUR FLORIANOPOLIS - REDE ANDRADE', 'HOTEL INTERCITY PORTOFINO FLORIANOPOLIS', 'HOTEL VALERIM FLORIANOPOLIS', 'SLAVIERO BAIA NORTE FLORIANOPOLIS', 'FAIAL PRIME SUITES', 'CASTELMAR HOTEL', 'BLUE TREE PREMIUM FLORIANOPOLIS', 'NOVOTEL FLORIANOPOLIS', 'CRIS HOTEL', 'VILLAGE PARAISO', 'LUMAR HOTEL - REDE ANDRADE', 'GENTIL HOTEL', 'HOTEL INTERCITY FLORIANOPOLIS', 'INGLESES PRAIA HOTEL', 'PORTO DA ILHA HOTEL', 'INTERCLASS FLORIANOPOLIS', 'IATE HOTEL CENTRO FLORIANOPOLIS', 'RIO BRANCO APART HOTEL', 'HOTEL PORTO SOL BEACH', 'MARIA DO MAR HOTEL', 'HOTEL BOUTIQUE QUINTA DAS VIDEIRAS', 'OSCAR HOTEL', 'HOTEL PALACE 1', 'JURERE BEACH VILLAGE', 'APART HOTEL ROBERTO MONTEIRO', 'INGLESES PARK HOTEL', 'IL CAMPANARIO VILLAGGIO RESORT', 'WK DESIGN HOTEL FLORIANOPOLIS', 'REDE ANDRADE FLOPH', 'IBIS FLORIANOPOLIS', 'HOTEL CANASVIEIRAS INTERNACIONAL - HCI', 'AL MARE PALACE HOTEL', "QUINTA DA BICA D' AGUA STAYHOTEL.CLUB", 'HOTEL TORRES DA CACHOEIRA', 'CARAVAGGIO PRAIA HOTEL', 'HOTEL ILHA BELA', 'DA

In [10]:
price_hotels_123m_fmtg = []

for price in list_price_hotels_123milhas:
    for dot in ["."]:
        price = price.replace(dot, "")
    price_hotels_123m_fmtg.append(price)
print(price_hotels_123m_fmtg)

['R$ 320', 'R$ 453', 'R$ 396', 'R$ 570', 'R$ 585', 'R$ 310', 'R$ 825', 'R$ 636', 'R$ 597', 'R$ 638', 'R$ 307', 'R$ 578', 'R$ 687', 'R$ 1166', 'R$ 679', 'R$ 517', 'R$ 829', 'R$ 721', 'R$ 777', 'R$ 448', 'R$ 1458', 'R$ 413', 'R$ 890', 'R$ 1533', 'R$ 732', 'R$ 650', 'R$ 2505', 'R$ 1070', 'R$ 307', 'R$ 300', 'R$ 344', 'R$ 344', 'R$ 522', 'R$ 1426', 'R$ 324', 'R$ 325', 'R$ 338', 'R$ 346', 'R$ 379', 'R$ 404', 'R$ 408', 'R$ 417', 'R$ 434', 'R$ 436', 'R$ 438', 'R$ 440', 'R$ 446', 'R$ 505', 'R$ 512', 'R$ 515', 'R$ 522', 'R$ 535', 'R$ 595', 'R$ 595', 'R$ 606', 'R$ 620', 'R$ 637', 'R$ 637', 'R$ 641', 'R$ 689', 'R$ 706', 'R$ 739', 'R$ 740', 'R$ 918', 'R$ 988', 'R$ 1069', 'R$ 1135', 'R$ 266', 'R$ 357', 'R$ 518', 'R$ 525', 'R$ 537', 'R$ 588', 'R$ 651', 'R$ 729', 'R$ 804', 'R$ 955', 'R$ 1142', 'R$ 314', 'R$ 357', 'R$ 373', 'R$ 651', 'R$ 697', 'R$ 769', 'R$ 787', 'R$ 823', 'R$ 854', 'R$ 918', 'R$ 1257', 'R$ 1308', 'R$ 233', 'R$ 312', 'R$ 357', 'R$ 365', 'R$ 439', 'R$ 464', 'R$ 526', 'R$ 537', 'R$ 564'

In [11]:
price_hotels_123m_fmtg2 = []

for price in price_hotels_123m_fmtg:
    for sign in ["R$ "]:
        price = price.replace(sign, "")
    price_hotels_123m_fmtg2.append(price)
print(price_hotels_123m_fmtg2)

['320', '453', '396', '570', '585', '310', '825', '636', '597', '638', '307', '578', '687', '1166', '679', '517', '829', '721', '777', '448', '1458', '413', '890', '1533', '732', '650', '2505', '1070', '307', '300', '344', '344', '522', '1426', '324', '325', '338', '346', '379', '404', '408', '417', '434', '436', '438', '440', '446', '505', '512', '515', '522', '535', '595', '595', '606', '620', '637', '637', '641', '689', '706', '739', '740', '918', '988', '1069', '1135', '266', '357', '518', '525', '537', '588', '651', '729', '804', '955', '1142', '314', '357', '373', '651', '697', '769', '787', '823', '854', '918', '1257', '1308', '233', '312', '357', '365', '439', '464', '526', '537', '564', '566', '599', '646', '682', '719', '758', '844', '564', '566', '599', '646', '682', '719', '758', '844', '564', '566', '599', '646', '682', '719', '758', '844', '564', '566', '599', '646', '682', '719', '758', '844', '564', '566', '599', '646', '682', '719', '758', '844', '564', '566', '599', '

In [12]:
price_hotels_123m_fmtg2 = pd.to_numeric(price_hotels_123m_fmtg2, errors='coerce', downcast='unsigned')

In [13]:
data_hoteis_123m = []

for elems in zip(name_hotels_123m_fmtg, price_hotels_123m_fmtg2):
    data_hoteis_123m.append(elems)
data_hoteis_123m_dict = dict(data_hoteis_123m)
print(data_hoteis_123m_dict)

{'HOTEL CECOMTUR FLORIANOPOLIS - REDE ANDRADE': 320, 'HOTEL INTERCITY PORTOFINO FLORIANOPOLIS': 453, 'HOTEL VALERIM FLORIANOPOLIS': 396, 'SLAVIERO BAIA NORTE FLORIANOPOLIS': 570, 'FAIAL PRIME SUITES': 585, 'CASTELMAR HOTEL': 310, 'BLUE TREE PREMIUM FLORIANOPOLIS': 825, 'NOVOTEL FLORIANOPOLIS': 636, 'CRIS HOTEL': 597, 'VILLAGE PARAISO': 638, 'LUMAR HOTEL - REDE ANDRADE': 307, 'GENTIL HOTEL': 578, 'HOTEL INTERCITY FLORIANOPOLIS': 687, 'INGLESES PRAIA HOTEL': 1166, 'PORTO DA ILHA HOTEL': 679, 'INTERCLASS FLORIANOPOLIS': 517, 'IATE HOTEL CENTRO FLORIANOPOLIS': 829, 'RIO BRANCO APART HOTEL': 721, 'HOTEL PORTO SOL BEACH': 777, 'MARIA DO MAR HOTEL': 448, 'HOTEL BOUTIQUE QUINTA DAS VIDEIRAS': 1458, 'OSCAR HOTEL': 413, 'HOTEL PALACE 1': 890, 'JURERE BEACH VILLAGE': 1533, 'APART HOTEL ROBERTO MONTEIRO': 732, 'INGLESES PARK HOTEL': 650, 'IL CAMPANARIO VILLAGGIO RESORT': 2505, 'WK DESIGN HOTEL FLORIANOPOLIS': 1070, 'REDE ANDRADE FLOPH': 307, 'IBIS FLORIANOPOLIS': 300, 'HOTEL CANASVIEIRAS INTERNACI

In [14]:
df_data_hoteis_123m = pd.DataFrame(list(data_hoteis_123m_dict.items()),
                   columns=['Hotel Name', 'Net Price 123 Milhas'])
display(df_data_hoteis_123m)

,Hotel Name,Net Price 123 Milhas
0,HOTEL CECOMTUR FLORIANOPOLIS - REDE ANDRADE,320
1,HOTEL INTERCITY PORTOFINO FLORIANOPOLIS,453
2,HOTEL VALERIM FLORIANOPOLIS,396
3,SLAVIERO BAIA NORTE FLORIANOPOLIS,570
4,FAIAL PRIME SUITES,585
...,...,...
101,KASATO MARU APART HOTEL,646
102,POUSADA ESTRELA GUIA,682
103,MARATEA MARE,719
104,PRAIA MOLE RESIDENCE,758


In [15]:
df_data_hoteis_123m['Total Price 123 Milhas (w/ fee)'] = (df_data_hoteis_123m['Net Price 123 Milhas'] * fee) + df_data_hoteis_123m['Net Price 123 Milhas'].apply(pd.to_numeric) 

In [16]:
df_data_hoteis_123m['Total Price 123 Milhas (w/ fee)'] = df_data_hoteis_123m['Total Price 123 Milhas (w/ fee)'].astype(int)

In [17]:
df_data_hoteis_123m

,Hotel Name,Net Price 123 Milhas,Total Price 123 Milhas (w/ fee)
0,HOTEL CECOMTUR FLORIANOPOLIS - REDE ANDRADE,320,366
1,HOTEL INTERCITY PORTOFINO FLORIANOPOLIS,453,518
2,HOTEL VALERIM FLORIANOPOLIS,396,453
3,SLAVIERO BAIA NORTE FLORIANOPOLIS,570,652
4,FAIAL PRIME SUITES,585,669
...,...,...,...
101,KASATO MARU APART HOTEL,646,739
102,POUSADA ESTRELA GUIA,682,780
103,MARATEA MARE,719,822
104,PRAIA MOLE RESIDENCE,758,867


In [18]:
length_df1 = len(df_data_hoteis_123m)
length_df2 = len(list_channel_hotels_123milhas)
diff_length_dfs = length_df2 - length_df1
list_channel_hotels_123milhas = list_channel_hotels_123milhas[:length_df2 - diff_length_dfs]

In [19]:
df_data_hoteis_123m.insert(loc=1, column='Omnibees Provider 123 Milhas', value=list_channel_hotels_123milhas)

In [20]:
df_data_hoteis_123m

,Hotel Name,Omnibees Provider 123 Milhas,Net Price 123 Milhas,Total Price 123 Milhas (w/ fee)
0,HOTEL CECOMTUR FLORIANOPOLIS - REDE ANDRADE,0,320,366
1,HOTEL INTERCITY PORTOFINO FLORIANOPOLIS,1,453,518
2,HOTEL VALERIM FLORIANOPOLIS,0,396,453
3,SLAVIERO BAIA NORTE FLORIANOPOLIS,1,570,652
4,FAIAL PRIME SUITES,0,585,669
...,...,...,...,...
101,KASATO MARU APART HOTEL,0,646,739
102,POUSADA ESTRELA GUIA,0,682,780
103,MARATEA MARE,0,719,822
104,PRAIA MOLE RESIDENCE,0,758,867


In [21]:
copy_df_123m = df_data_hoteis_123m.copy()

In [22]:
from datetime import datetime

checkin_decolar = datetime.strptime(checkin, "%d-%m-%Y").strftime('%Y-%m-%d')
checkout_decolar = datetime.strptime(checkout, "%d-%m-%Y").strftime('%Y-%m-%d')

In [23]:
# Scraper Hotel Prices Decolar

list_name_hotels_decolar = []
list_price_hotels_decolar = []

class ScraperDecolar:

    def scraper(self):
        self.data_collect()

    def data_collect(self): 
        page = 1      
        self.driver = webdriver.Chrome(options=chrome_options)
        self.link = f'https://www.decolar.com/accommodations/results/CIT_{id_destination_decolar}/{checkin_decolar}/{checkout_decolar}/{pax}?from=SB2&facet=city&searchId=2c403941-a9b6-457f-a17e-5f3131886e89&page={page}'
        self.list_names = []
        self.list_prices = []
        self.driver.get(self.link)
        self.driver.find_element(By.XPATH, '//*[@id="lgpd-banner"]/div/a[2]').click()
        sleep(3)
        

        for p in range(20): # Number of pages to browse (nº > total, run through all)
            item = 1
            list_length = self.driver.find_elements(By.CLASS_NAME, 'accommodation-name')
            list_max = len(list_length)
            for i in range(list_max):
                c = 1
                while c < list_max:
                    try:
                        names_temp = self.driver.find_elements(By.XPATH,
                            f'/html/body/aloha-app-root/aloha-results/div/div/div/div/div[2]/aloha-list-view-container/div[2]/div[{item}]/aloha-cluster-container/div/div/div[1]/div/div[2]/div/aloha-cluster-accommodation-info-container/div[1]/span')
                        self.list_names.append(names_temp[0].text)
                        sleep(1)
                        prices_temp = self.driver.find_elements(By.XPATH,
                            f'/html/body/aloha-app-root/aloha-results/div/div/div/div/div[2]/aloha-list-view-container/div[2]/div[{item}]/aloha-cluster-container/div/div/div[2]/aloha-cluster-pricebox-container/div/div[2]/div[1]/aloha-price-container/aloha-summary-price/div/span[2]')
                        self.list_prices.append(prices_temp[0].text)          
                        sleep(1)
                        item += 1
                    except:
                        item += 1
                        c += 1                        
            try:
                page += 1
                self.driver.get(f'https://www.decolar.com/accommodations/results/CIT_{id_destination_decolar}/{checkin_decolar}/{checkout_decolar}/{pax}?from=SB2&facet=city&searchId=2c403941-a9b6-457f-a17e-5f3131886e89&page={page}')
                print(f'\u001b[32m{"Browsing..."}\u001b[0m')
                sleep(5)
            except:
                self.driver.quit()
        
        print("Raw data:")
        print(self.list_names)
        print(self.list_prices)

        for name in self.list_names:
            list_name_hotels_decolar.append(name)
        for price in self.list_prices:
            list_price_hotels_decolar.append(price)


start = ScraperDecolar()
start.scraper()


Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Raw data:
['Al Mare Palace', 'Hotel São Sebastião da Praia', 'Hotel & Pousada Favareto', 'Pousada Oceanomare', 'Hotel Porto Sol Beach', 'Jurerê Beach Village', 'Intercity Florianópolis', 'HCI Hotel Florianópolis', 'Hotel Porto Sol Ingleses', 'Ingleses Park Hotel', 'Das Nações Hotel', 'Rede Andrade Cecomtur', 'Majestic Palace', 'Hotel Ingleses Holiday', 'Pousada Vó Regina', 'Hotel Praia Brava', 'Belluno Apart Hotel', 'Varadero Palace Hotel I', 'WK Design Hotel Florianópolis', 'Interclass Florianópolis', 'Village Paraíso', 'Novotel Florianópolis', 'Palm Beach Apart Hotel', 'Hotel Monteiro Canasvieiras', 'Ilha Norte Apart Hotel', 'Blue Tree Premium Florianópolis', 'IL Campanario Villaggio Resort', 'Torres da Cachoeira', 'Castelmar Hotel', 'Gaivotas Prai

In [24]:
# Data cleaning

name_hotels_desp_temp = list(pd.Series(list_name_hotels_decolar).str.upper())
name_hotels_desp_fmtg = []

for name in name_hotels_desp_temp:
    name_hotels_desp_fmtg.append(unidecode(name))
print(name_hotels_desp_fmtg)

['AL MARE PALACE', 'HOTEL SAO SEBASTIAO DA PRAIA', 'HOTEL & POUSADA FAVARETO', 'POUSADA OCEANOMARE', 'HOTEL PORTO SOL BEACH', 'JURERE BEACH VILLAGE', 'INTERCITY FLORIANOPOLIS', 'HCI HOTEL FLORIANOPOLIS', 'HOTEL PORTO SOL INGLESES', 'INGLESES PARK HOTEL', 'DAS NACOES HOTEL', 'REDE ANDRADE CECOMTUR', 'MAJESTIC PALACE', 'HOTEL INGLESES HOLIDAY', 'POUSADA VO REGINA', 'HOTEL PRAIA BRAVA', 'BELLUNO APART HOTEL', 'VARADERO PALACE HOTEL I', 'WK DESIGN HOTEL FLORIANOPOLIS', 'INTERCLASS FLORIANOPOLIS', 'VILLAGE PARAISO', 'NOVOTEL FLORIANOPOLIS', 'PALM BEACH APART HOTEL', 'HOTEL MONTEIRO CANASVIEIRAS', 'ILHA NORTE APART HOTEL', 'BLUE TREE PREMIUM FLORIANOPOLIS', 'IL CAMPANARIO VILLAGGIO RESORT', 'TORRES DA CACHOEIRA', 'CASTELMAR HOTEL', 'GAIVOTAS PRAIA HOTEL', 'GECKOS HOSTEL', 'HOTEL INTERCITY PORTOFINO FLORIANOPOLIS', 'HOTEL FAROL DA ILHA', 'POUSADA DOS CHAS HOTEL BOUTIQUE', 'ELITE HOTEL', 'VILLAS JURERE HOTEL BOUTIQUE', 'MERCURE FLORIANOPOLIS', 'FIRENZE BUSINESS HOTEL', 'MARATEA MARE HOTEL', 'M

In [25]:
price_hoteis_desp_fmtg = []

for price in list_price_hotels_decolar:
    for dot in ["."]:
        price = price.replace(dot, "")
    price_hoteis_desp_fmtg.append(price)
print(price_hoteis_desp_fmtg)

['476', '1012', '940', '626', '1180', '2208', '940', '478', '898', '962', '502', '374', '1130', '1124', '618', '984', '794', '778', '1500', '664', '988', '854', '814', '1032', '422', '1220', '3348', '1594', '444', '722', '372', '670', '364', '1202', '640', '1480', '654', '536', '964', '600', '850', '568', '672', '864', '628', '566', '602', '680', '484', '576', '758', '1102', '662', '484', '642', '1934', '1000', '520', '502', '706', '582', '606', '782', '484', '422', '2800', '762', '902', '322', '1244', '802', '492', '572', '536', '590', '660', '366', '562', '792', '468', '904', '816', '708', '410', '674', '1274', '732', '418', '1022', '270', '402', '764', '514', '4102', '386', '312', '1618', '574', '602', '344', '782', '782', '864', '622', '892', '1352', '482', '1820', '544', '1584', '288', '1414', '320', '964', '822', '614', '1064', '372', '622', '562', '662', '646', '374', '558', '312', '412', '800', '438', '412', '842', '770', '716', '842', '756', '238', '412', '502', '1048', '614',

In [26]:
price_hoteis_desp_fmtg = pd.to_numeric(price_hoteis_desp_fmtg, errors='coerce')

In [27]:
data_hotels_desp = []

for elems in zip(name_hotels_desp_fmtg, price_hoteis_desp_fmtg):
    data_hotels_desp.append(elems)
data_hotels_desp_dict = dict(data_hotels_desp)
print(data_hotels_desp_dict)

{'AL MARE PALACE': 476, 'HOTEL SAO SEBASTIAO DA PRAIA': 1012, 'HOTEL & POUSADA FAVARETO': 940, 'POUSADA OCEANOMARE': 626, 'HOTEL PORTO SOL BEACH': 1180, 'JURERE BEACH VILLAGE': 2208, 'INTERCITY FLORIANOPOLIS': 940, 'HCI HOTEL FLORIANOPOLIS': 478, 'HOTEL PORTO SOL INGLESES': 898, 'INGLESES PARK HOTEL': 962, 'DAS NACOES HOTEL': 502, 'REDE ANDRADE CECOMTUR': 374, 'MAJESTIC PALACE': 1130, 'HOTEL INGLESES HOLIDAY': 1124, 'POUSADA VO REGINA': 618, 'HOTEL PRAIA BRAVA': 984, 'BELLUNO APART HOTEL': 794, 'VARADERO PALACE HOTEL I': 778, 'WK DESIGN HOTEL FLORIANOPOLIS': 1500, 'INTERCLASS FLORIANOPOLIS': 664, 'VILLAGE PARAISO': 988, 'NOVOTEL FLORIANOPOLIS': 854, 'PALM BEACH APART HOTEL': 814, 'HOTEL MONTEIRO CANASVIEIRAS': 1032, 'ILHA NORTE APART HOTEL': 422, 'BLUE TREE PREMIUM FLORIANOPOLIS': 1220, 'IL CAMPANARIO VILLAGGIO RESORT': 3348, 'TORRES DA CACHOEIRA': 1594, 'CASTELMAR HOTEL': 444, 'GAIVOTAS PRAIA HOTEL': 722, 'GECKOS HOSTEL': 372, 'HOTEL INTERCITY PORTOFINO FLORIANOPOLIS': 670, 'HOTEL FAR

In [28]:
df_data_hotels_desp = pd.DataFrame(list(data_hotels_desp_dict.items()),
                   columns=['Hotel Name', 'Total Price Decolar'])
display(df_data_hotels_desp)

,Hotel Name,Total Price Decolar
0,AL MARE PALACE,476
1,HOTEL SAO SEBASTIAO DA PRAIA,1012
2,HOTEL & POUSADA FAVARETO,940
3,POUSADA OCEANOMARE,626
4,HOTEL PORTO SOL BEACH,1180
...,...,...
188,FLORIANOPOLIS VACATION APARTMENT RENTAL,848
189,CASA DO RIO VERMELHO- WITH JACUZZI,2148
190,FLAT KAKI IN NEW CAMPECHE SOUTH ISLAND,702
191,PARADISE MOUNTAIN BEACH BUNGALOW,702


In [29]:
copy_df_desp = df_data_hotels_desp.copy()

In [32]:
# Data merge

from pyjarowinkler import distance

for hotelName123m in df_data_hoteis_123m['Hotel Name']:
  for hotelNameDesp in df_data_hotels_desp['Hotel Name']:
    similarity = distance.get_jaro_distance(hotelName123m, hotelNameDesp, winkler=True, scaling=0.1)
    if similarity > 0.75:
      df_data_hoteis_123m['Hotel Name'] = df_data_hotels_desp['Hotel Name']

In [33]:
data_merge = pd.merge(df_data_hoteis_123m, df_data_hotels_desp, on=['Hotel Name'], how='left')
display(data_merge)

,Hotel Name,Omnibees Provider 123 Milhas,Net Price 123 Milhas,Total Price 123 Milhas (w/ fee),Total Price Decolar
0,AL MARE PALACE,0,320,366,476
1,HOTEL SAO SEBASTIAO DA PRAIA,1,453,518,1012
2,HOTEL & POUSADA FAVARETO,0,396,453,940
3,POUSADA OCEANOMARE,1,570,652,626
4,HOTEL PORTO SOL BEACH,0,585,669,1180
...,...,...,...,...,...
101,POUSADA DOS GOLFINHOS,0,646,739,864
102,POUSADA KINDERMANN,0,682,780,622
103,CABANAS PRAIA MOLE,0,719,822,892
104,HAUTE HAUS,0,758,867,1352


In [34]:
df_rate_comparison = data_merge

In [41]:
for prices in df_rate_comparison:
    diff = (df_rate_comparison['Total Price Decolar'] / df_rate_comparison['Total Price 123 Milhas (w/ fee)'] -1 ) * 100
    df_rate_comparison['% Variation'] = round(diff)
display(df_rate_comparison)

,Hotel Name,Omnibees Provider 123 Milhas,Net Price 123 Milhas,Total Price 123 Milhas (w/ fee),Total Price Decolar,% Variation
0,AL MARE PALACE,0,320,366,476,30.0
1,HOTEL SAO SEBASTIAO DA PRAIA,1,453,518,1012,95.0
2,HOTEL & POUSADA FAVARETO,0,396,453,940,108.0
3,POUSADA OCEANOMARE,1,570,652,626,-4.0
4,HOTEL PORTO SOL BEACH,0,585,669,1180,76.0
...,...,...,...,...,...,...
101,POUSADA DOS GOLFINHOS,0,646,739,864,17.0
102,POUSADA KINDERMANN,0,682,780,622,-20.0
103,CABANAS PRAIA MOLE,0,719,822,892,9.0
104,HAUTE HAUS,0,758,867,1352,56.0


In [36]:
data_merge.fillna('-')

,Hotel Name,Omnibees Provider 123 Milhas,Net Price 123 Milhas,Total Price 123 Milhas (w/ fee),Total Price Decolar,% Variation
0,AL MARE PALACE,0,320,366,476,30.0
1,HOTEL SAO SEBASTIAO DA PRAIA,1,453,518,1012,95.0
2,HOTEL & POUSADA FAVARETO,0,396,453,940,108.0
3,POUSADA OCEANOMARE,1,570,652,626,-4.0
4,HOTEL PORTO SOL BEACH,0,585,669,1180,76.0
...,...,...,...,...,...,...
101,POUSADA DOS GOLFINHOS,0,646,739,864,17.0
102,POUSADA KINDERMANN,0,682,780,622,-20.0
103,CABANAS PRAIA MOLE,0,719,822,892,9.0
104,HAUTE HAUS,0,758,867,1352,56.0


In [37]:
# Summary

import math

measurements = 0
for m in df_rate_comparison['% Variation']:
  if not math.isnan(m):
    measurements += 1
print(f'Total measurements: {measurements}')

unfavorable = 0
for u in df_rate_comparison['% Variation']:
    if u >= 1:
        unfavorable += 1
percent_unfavorable = (unfavorable / measurements) * 100
print(f'Percentage of unfavorable measurements: {percent_unfavorable:.2f}%')

Total measurements: 106
Percentage of unfavorable measurements: 51.89%


In [38]:
# Highlights

def highlight(s):
    color_red = 'red'
    color_yellow = 'yellow'
    color_green = 'green'
    if s >= 1:
      return 'background-color: %s' % color_red
    elif s < 1 and s > -1:
      return 'background-color: %s' % color_yellow
    elif s <= -1:
      return 'background-color: %s' % color_green

report = df_rate_comparison.style.applymap(highlight, subset=pd.IndexSlice[:, ['% Variation']])

In [39]:
# Dataframe to XLSX

writer = pd.ExcelWriter('Rate-parity-report-123milhas-Decolar-{}.xlsx'.format(name_destination))
report.to_excel(writer, f'IN_{checkin}-OUT_{checkout}', index=False)
writer.save()

In [40]:
# Download XLSX

from google.colab import files

files.download('Rate-parity-report-123milhas-Decolar-{}.xlsx'.format(name_destination))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>